# 実験を途中保存する

実験終了前の段階で実験をデータガバナンス機能に途中保存します。  
上から順番に実行してください。

## 1. コミットメッセージを変数に入力する

以下のセルを実行して、実験作業のログとして残す短いメッセージを入力してください。  
※入力値に誤りがある場合、もう一度実行することで訂正ができます。

In [ ]:
message = input()

## 2. 高性能実験環境から実験記録を取得する

高性能実験環境を利用している場合には、以下のセルを実行してください。

In [ ]:
import os
tmp = os.environ["BINDER_REF_URL"]
tmp = tmp.split("/")
tmp = tmp[len(tmp) - 1]
repo_name = tmp[:tmp.find(".")]

!rm ~/input_data/*
!rm ~/output_data/*
!rm ~/source/*

!scp mdx:~/$repo_name/input_data/* ~/input_data/
!scp mdx:~/$repo_name/output_data/* ~/output_data/
!scp mdx:~/$repo_name/source/* ~/source/

## 3. 実験をデータガバナンス機能に途中保存する

In [ ]:
from IPython.display import display, Javascript
display(Javascript('IPython.notebook.save_checkpoint();'))

In [ ]:
import os
import glob

# Git管理のパスのリストを作成する
%cd ~/
files = os.listdir()
# ディレクトリ一覧からGit-annex管理するディレクトリ(input_dataとoutput_data)を排除する
dirs = [f for f in files if os.path.isdir(f)]
dirs.remove('input_data')
dirs.remove('output_data')
# HOME直下のファイルを取得
files = [f for f in files if os.path.isfile(f)]
# Git管理するパスの配列を作成する
files.extend(dirs)
save_path = files

# Git-annex管理するパスの配列を作成する
annexed_save_path = ['input_data', 'output_data']

In [ ]:
import papermill as pm

%cd ~/
# Git-annex管理ファイルを保存
pm.execute_notebook(
    'EX-WORKFLOW/util/base_datalad_save_push.ipynb',
    '/home/jovyan/.local/push_log.ipynb',
    parameters = dict(SAVE_MESSAGE = message, PATH = annexed_save_path, IS_RECURSIVE = False)
)
# Git管理ファイルを保存
pm.execute_notebook(
    'EX-WORKFLOW/util/base_datalad_save_push.ipynb',
    '/home/jovyan/.local/push_log.ipynb',
    parameters = dict(SAVE_MESSAGE = message, TO_GIT = True, PATH = save_path, IS_RECURSIVE = False)
)

## 4. 実験中ワークフロー機能トップページに遷移する

続けてワークフロー機能を実行する場合は、[こちら](../experiment.ipynb)からトップページに遷移できます。  